In [1]:
!pip install -U "openai>=1.0.0"
import sys

# Add the directory to the Python path
sys.path.append('/Users/matt/code/kamiwaza')

# You can now import modules from that directory


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import logging
import socket
import re
import platform
import time
from typing import Optional, Dict, Any

# Install latest openai
import openai

from kamiwaza.serving.services import ServingService

local_model = None

# Set up logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger("requests").setLevel(logging.WARNING)
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name).setLevel(logging.WARNING)

def get_deployed_model(prefer_code: bool = True) -> Dict[str, Any]:
    """
    Find a deployed model, preferring ones with 'code' in the name if specified.
    
    Args:
        prefer_code: Whether to prefer models with 'code' in the name
        
    Returns:
        Dict containing deployment info with host, port and model name
        
    Raises:
        ValueError: If no suitable deployment is found
    """
    serving_service = ServingService()
    deployments = serving_service.list_deployments(with_names=True)
    
    if not deployments:
        raise ValueError("No deployments found")
        
    # First try to find a deployed code model if preferred
    if prefer_code:
        for deployment in deployments:
            if (deployment.status == 'DEPLOYED' and 
                deployment.instances and
                'code' in deployment.m_name.lower()):
                return {
                    'host': socket.gethostname(),
                    'port': deployment.lb_port,
                    'name': deployment.m_name
                }
    
    # Fall back to first deployed model
    for deployment in deployments:
        if deployment.status == 'DEPLOYED' and deployment.instances:
            return {
                'host': socket.gethostname(),
                'port': deployment.lb_port,
                'name': deployment.m_name
            }
            
    raise ValueError("No deployed models found")

def infer(prompt: str, model_info: Optional[Dict[str, Any]] = None) -> str:
    """
    Run inference on a deployed model.
    
    Args:
        prompt: The input prompt
        model_info: Optional dict with model deployment info. If None, will scan for a model.
        
    Returns:
        The model's response as plain text
        
    Raises:
        ValueError: If no model is available or inference fails
    """
    if model_info is None:
        global local_model
        if local_model:
            model_info = local_model
        else:
            model_info = get_deployed_model()
            if not model_info:
                print("### No MODEL!")
                return -1
            local_model = model_info
        
    client = openai.OpenAI(
        base_url=f"http://{model_info['host']}:{model_info['port']}/v1",
        api_key="not-needed"
    )
    
    try:
        response = client.chat.completions.create(
            model=model_info['name'],
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=1024
        )
        return response.choices[0].message.content.strip()
        
    except Exception as e:
        raise ValueError(f"Inference failed: {str(e)}") from e

# Example usage:
# model = get_deployed_model()
# response = infer("Write a function to calculate fibonacci numbers")
# print(response)



In [5]:
model = get_deployed_model()
print(infer("say hi"))

Hi there! How can I assist you today?


In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Define the file path
file_path = '~/Downloads/beedata.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Check if the first row is the header
header = df.columns.tolist()
print("Header:", header)

# Print the first 5 rows
print("\nFirst 5 rows:")
print(df.head(5))